### Import and Load Images

In [1]:
import numpy as np
import cv2
import  matplotlib.pyplot as plt
import os
import sys
sys.path.append('../Scripts/')
from Scripts.Utility import Utility2
ut = Utility2()

root_dir = os.path.abspath("../")
imagePath = "/Input/Images"

in init


In [2]:
images = ut.loadImages(imagePath)
#ut.showImages(images, scale=(500,500))

### Apply processing
1. Gabor
2. Histogram Equalization
3. Median Smooth

In [3]:
smoothed_eq_img_arr, smt_arr, f_img, f_values = ut.processImage(images[0])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--BFP.tif")
smoothed_eq_img_arr1, smt_arr1, f_img1, f_values1 = ut.processImage(images[1])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--DsRed.tif")
smoothed_eq_img_arr2, smt_arr2, f_img2, f_values2 = ut.processImage(images[2])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--eGFP.tif")
smoothed_eq_img_arr3, smt_arr3, f_img3, f_values3 = ut.processImage(images[3])#ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--miRFP.tif")

In [4]:
pairs = ut.pairs_from_array(smoothed_eq_img_arr, smt_arr)
#ut.showImages(pairs, scale=(1000,500))
#ut.writeImages(pairs, path="../ImageWrite")

### Merge Processed Images (Single Pair)
Gamma = 0.25

Theta = pi/4 && 3*pi/4

In [5]:
#smts = [smoothed_eq_img_arr,smoothed_eq_img_arr1, smoothed_eq_img_arr2,smoothed_eq_img_arr3]
#mergedImages = ut.mergeImages(smts, 0,3, 0.5)

mergedImage  = cv2.addWeighted(smoothed_eq_img_arr[1], 0.5, smoothed_eq_img_arr[5], .5, 0)
mergedImage1  = cv2.addWeighted(smoothed_eq_img_arr1[1], 0.5, smoothed_eq_img_arr1[5], .5, 0)
mergedImage2  = cv2.addWeighted(smoothed_eq_img_arr2[1], 0.5, smoothed_eq_img_arr2[5], .5, 0)
mergedImage3  = cv2.addWeighted(smoothed_eq_img_arr3[1], 0.5, smoothed_eq_img_arr3[5], .5, 0)
#cv2.imshow(str("mergedImage"), mergedImage)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#ut.showImages(smoothed_eq_img_arr, scale=(500,500))

### Watershed Process

In [7]:
MergedImages = [mergedImage, mergedImage1, mergedImage2, mergedImage3]
GrayMergedArr = ut.grayscaleImages(MergedImages)

ThresholdArr = ut.thresholdImages(GrayMergedArr)
KmeansApplied = ut.kmeansOutImages(MergedImages, 5)
output, markers = ut.waterShedImages(KmeansApplied)


ut.showImages(output, scale=(500,500))

#GrayMerged = cv2.cvtColor(mergedImage,cv2.COLOR_BGR2GRAY)
#GrayMerged1 = cv2.cvtColor(mergedImage1,cv2.COLOR_BGR2GRAY)
#GrayMerged2 = cv2.cvtColor(mergedImage2,cv2.COLOR_BGR2GRAY)
#GrayMerged3 = cv2.cvtColor(mergedImage3,cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(GrayMerged,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#ret1, thresh1 = cv2.threshold(GrayMerged1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#ret2, thresh2 = cv2.threshold(GrayMerged2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#ret3, thresh3 = cv2.threshold(GrayMerged3,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

